In [74]:
import json
import pandas as pd

# Ruta al archivo que contiene el JSON
ruta_archivo = '../PI_ML_OPS_data util/output_steam_games.json'

# Lista para almacenar los objetos JSON
lista_objetos_json = []

# Leer el contenido del archivo línea por línea
with open(ruta_archivo, 'r',encoding="utf-8") as f:
    for linea in f:
        try:
            # Utilizar json.loads para cargar cada línea como un objeto JSON
            objeto_json = json.loads(linea)
            lista_objetos_json.append(objeto_json)
        except json.JSONDecodeError as e:
            print(f"Error al decodificar el JSON en la línea: {linea.strip()}. {e}")

# Convierte la lista de objetos a un DataFrame de Pandas para mejor lectura
data = pd.DataFrame(lista_objetos_json)




In [75]:
# Eliminar filas con valores NaN
# Solo aquellas que tengan toda la fila vacía
data_cleaned = data.dropna(how='all')

# Mostrar los primeros registros para verificar la lectura
print(data_cleaned.head())

              publisher                                             genres  \
88310         Kotoshiro      [Action, Casual, Indie, Simulation, Strategy]   
88311  Making Fun, Inc.               [Free to Play, Indie, RPG, Strategy]   
88312      Poolians.com  [Casual, Free to Play, Indie, Simulation, Sports]   
88313              彼岸领域                        [Action, Adventure, Casual]   
88314               NaN                                                NaN   

                      app_name                    title  \
88310      Lost Summoner Kitty      Lost Summoner Kitty   
88311                Ironbound                Ironbound   
88312  Real Pool 3D - Poolians  Real Pool 3D - Poolians   
88313                  弹炸人2222                  弹炸人2222   
88314            Log Challenge                      NaN   

                                                     url release_date  \
88310  http://store.steampowered.com/app/761140/Lost_...   2018-01-04   
88311  http://store.steampower

In [76]:
# Reviso cantidad de nulos iniciales
print(data.isna().sum())

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64


In [77]:
# Reviso cantidad de nulos luego de dropna
print(data_cleaned.isna().sum())

publisher       8052
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64


In [78]:
# Elimino columnas que según análisis de diccionario no serán necesarias para los futuros endpoints
columns_to_drop = ['publisher', 'url', 'reviews_url', 'early_access']

data_cleaned = data_cleaned.drop(columns=columns_to_drop)


In [79]:
# Obtengo la composición actual del dataframe
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32135 entries, 88310 to 120444
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   genres        28852 non-null  object
 1   app_name      32133 non-null  object
 2   title         30085 non-null  object
 3   release_date  30068 non-null  object
 4   tags          31972 non-null  object
 5   specs         31465 non-null  object
 6   price         30758 non-null  object
 7   id            32133 non-null  object
 8   developer     28836 non-null  object
dtypes: object(9)
memory usage: 2.5+ MB


In [80]:
# Eliminar filas con nulos en 'app_name' y 'release_date'
data_cleaned = data_cleaned.dropna(subset=['app_name', 'release_date'])

# Verificar el resultado
print(data_cleaned.info())

<class 'pandas.core.frame.DataFrame'>
Index: 30067 entries, 88310 to 120443
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   genres        28833 non-null  object
 1   app_name      30067 non-null  object
 2   title         30067 non-null  object
 3   release_date  30067 non-null  object
 4   tags          29906 non-null  object
 5   specs         29398 non-null  object
 6   price         28821 non-null  object
 7   id            30066 non-null  object
 8   developer     28818 non-null  object
dtypes: object(9)
memory usage: 2.3+ MB
None


In [81]:
# Imprimir el tipo de dato de la columna 'price'
print(data_cleaned['price'].dtype)

object


In [82]:
# Contar el número de filas donde la columna 'price' es igual a 0
count_zeros = (data_cleaned['price'] == 0).sum()

# Imprimir el resultado
print(f"Número de filas con 'price' igual a 0: {count_zeros}")

Número de filas con 'price' igual a 0: 0


In [83]:
# Reemplazar valores específicos en la columna 'price'
data_cleaned.loc[data_cleaned['price'].isin(['Free to play', 'Free to use']), 'price'] = 0

# Convertir la columna 'price' a tipo numérico
data_cleaned.loc[:, 'price'] = pd.to_numeric(data_cleaned['price'], errors='coerce')

# Imprimir información sobre los tipos de datos después de la conversión
print(data_cleaned.dtypes)


genres          object
app_name        object
title           object
release_date    object
tags            object
specs           object
price           object
id              object
developer       object
dtype: object


In [84]:
# Reemplazar NaN por 0 solo en la columna 'price'
data_cleaned.loc[:,'price'].fillna(0)

88310     4.99
88311     0.00
88312     0.00
88313     0.99
88315     3.99
          ... 
120439    1.99
120440    1.99
120441    4.99
120442    1.99
120443    4.99
Name: price, Length: 30067, dtype: float64

In [85]:
filtered_data = data_cleaned[data_cleaned['price'] == 0]
print(filtered_data['developer'])

Series([], Name: developer, dtype: object)


In [86]:
# Verifica si hay duplicados en la columna 'title'
duplicates = data_cleaned['title'].duplicated()

# Muestra las filas que tienen títulos duplicados
duplicated_rows = data_cleaned[duplicates]
print(duplicated_rows)

                                          genres  \
89819                 [Adventure, Casual, Indie]   
100232  [Indie, Massively Multiplayer, Strategy]   
101963                [Action, Adventure, Indie]   
102883                                  [Action]   
105108                     [Adventure, Strategy]   
106374   [Action, Adventure, Casual, Indie, RPG]   
107374                                  [Casual]   
108615                      [Casual, Simulation]   
108661                                   [Indie]   
109299          [Adventure, Free to Play, Indie]   
109300                 [Adventure, Free to Play]   
109458                                  [Casual]   
110210                              [Simulation]   
111127                                       NaN   
111706                                       NaN   
112212          [Adventure, Free to Play, Indie]   
112716                [Action, Adventure, Indie]   
113877                           [Action, Indie]   
113955      

In [89]:
# Convertir 'release_date' a formato de fecha y obtener el año
data_cleaned['release_date'] = pd.to_datetime(data_cleaned['release_date'], errors='coerce')
data_cleaned['year'] = data_cleaned['release_date'].dt.year

# Eliminar la columna 'release_date' original
data_cleaned.drop(columns=['release_date'], inplace=True)

# Mostrar información del DataFrame resultante
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30067 entries, 88310 to 120443
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   genres     28833 non-null  object 
 1   app_name   30067 non-null  object 
 2   title      30067 non-null  object 
 3   tags       29906 non-null  object 
 4   specs      29398 non-null  object 
 5   price      27242 non-null  object 
 6   id         30066 non-null  object 
 7   developer  28818 non-null  object 
 8   year       29782 non-null  float64
dtypes: float64(1), object(8)
memory usage: 2.3+ MB


In [91]:
# Guardar el DataFrame modificado en un archivo CSV
data_cleaned.to_csv('steam_games.csv', index=False)

In [24]:
data_cleaned['id'].duplicated().any()

True

In [2]:
import pandas as pd
# Cargar los datos
data = pd.read_csv('steam_games.csv')

# Mostrar las primeras filas del DataFrame
print("Primeras filas del DataFrame:")
print(data.head())

Primeras filas del DataFrame:
                                              genres                 app_name  \
0  ['Action', 'Casual', 'Indie', 'Simulation', 'S...      Lost Summoner Kitty   
1       ['Free to Play', 'Indie', 'RPG', 'Strategy']                Ironbound   
2  ['Casual', 'Free to Play', 'Indie', 'Simulatio...  Real Pool 3D - Poolians   
3                  ['Action', 'Adventure', 'Casual']                  弹炸人2222   
4              ['Action', 'Adventure', 'Simulation']    Battle Royale Trainer   

                     title                                               tags  \
0      Lost Summoner Kitty  ['Strategy', 'Action', 'Indie', 'Casual', 'Sim...   
1                Ironbound  ['Free to Play', 'Strategy', 'Indie', 'RPG', '...   
2  Real Pool 3D - Poolians  ['Free to Play', 'Simulation', 'Sports', 'Casu...   
3                  弹炸人2222                  ['Action', 'Adventure', 'Casual']   
4    Battle Royale Trainer  ['Action', 'Adventure', 'Simulation', 'FPS', '... 

In [3]:
# Resumen estadístico
print("\nResumen estadístico:")
print(data.describe())



Resumen estadístico:
              price            id          year
count  27242.000000  3.006600e+04  29782.000000
mean       9.367760  4.439712e+05   2014.759183
std       16.461657  1.842216e+05      3.508268
min        0.490000  1.000000e+01   1970.000000
25%        2.990000  3.218325e+05   2014.000000
50%        4.990000  4.382650e+05   2016.000000
75%        9.990000  5.888700e+05   2017.000000
max      995.000000  2.028850e+06   2021.000000


In [4]:
# Tablas de frecuencia para variables categóricas
categoricas = data.select_dtypes(include=['object'])
for columna in categoricas.columns:
    print(f"\nTabla de frecuencia para {columna}:")
    print(data[columna].value_counts())



Tabla de frecuencia para genres:
genres
['Action']                                                                      1878
['Action', 'Indie']                                                             1648
['Simulation']                                                                  1396
['Casual', 'Simulation']                                                        1359
['Action', 'Adventure', 'Indie']                                                1081
                                                                                ... 
['Action', 'Adventure', 'Racing', 'Simulation', 'Strategy']                        1
['Action', 'Adventure', 'Casual', 'Indie', 'Racing', 'Sports', 'Strategy']         1
['Action', 'Adventure', 'Casual', 'Indie', 'Racing', 'Simulation', 'Sports']       1
['Action', 'Massively Multiplayer', 'RPG', 'Strategy']                             1
['Adventure', 'Casual', 'RPG', 'Simulation', 'Early Access']                       1
Name: count, Length: 882

In [5]:
# Análisis de valores perdidos
print("\nAnálisis de valores perdidos:")
print(data.isnull().sum())


Análisis de valores perdidos:
genres       1234
app_name        0
title           0
tags          161
specs         669
price        2825
id              1
developer    1249
year          285
dtype: int64
